In [16]:
import pandas as pd
import numpy as np
import uuid
import random

In [121]:
def make_node():
    return str(uuid.uuid4())

def make_int(null_prob=0.1):
    n = np.random.randint(1000)

    if np.random.rand() <= null_prob:
        n = None

    return n

def make_float(null_prob=0.1):
    n = np.random.rand()*100

    if np.random.rand() <= null_prob:
        n = None

    return n

def make_list(null_prob=0.1):
    n = np.random.randint(4)
    out = [make_node() for _ in range(n)]

    if np.random.rand() <= null_prob:
        out = None

    return out

def make_dict(null_prob=0.1):
    keys = list('ABCD')
    random.shuffle(keys)

    n = np.random.randint(len(keys))
    keys = keys[:n]

    d = {
        k: np.random.rand()
        for k in keys
    }

    if np.random.rand() <= null_prob:
        d = None

    return d

def make_category(null_prob=0.1):
    keys = list('ABCDEFG')
    n = np.random.randint(len(keys))
    key = keys[n]

    if np.random.rand() <= null_prob:
        key = None

    return key

In [127]:
make_category()

'G'

In [76]:
make_float()

2.5798608951148716

In [11]:
make_list()

['4e680029-b26a-43e1-995d-b9ae09f8feb6',
 '9f9a8afd-1a0c-4f80-9cf9-b9fddeecc33a']

In [128]:
def foo():
    d = {
        'node_id': make_node(),
        'the_int': make_int(),
        'the_float': make_float(),
        'the_list': make_list(),
        'the_dict': make_dict(),
        'the_category': make_category(),
    }
    return d

In [129]:
df = pd.DataFrame.from_records([foo() for _ in range(1000)])

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   node_id       1000 non-null   object 
 1   the_int       916 non-null    float64
 2   the_float     903 non-null    float64
 3   the_list      906 non-null    object 
 4   the_dict      908 non-null    object 
 5   the_category  910 non-null    object 
dtypes: float64(2), object(4)
memory usage: 47.0+ KB


In [131]:
df.infer_objects().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   node_id       1000 non-null   object 
 1   the_int       916 non-null    float64
 2   the_float     903 non-null    float64
 3   the_list      906 non-null    object 
 4   the_dict      908 non-null    object 
 5   the_category  910 non-null    object 
dtypes: float64(2), object(4)
memory usage: 47.0+ KB


In [133]:
df.convert_dtypes(dtype_backend='pyarrow').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   node_id       1000 non-null   string[pyarrow]
 1   the_int       916 non-null    int64[pyarrow] 
 2   the_float     903 non-null    double[pyarrow]
 3   the_list      906 non-null    object         
 4   the_dict      908 non-null    object         
 5   the_category  910 non-null    string[pyarrow]
dtypes: double[pyarrow](1), int64[pyarrow](1), object(2), string[pyarrow](2)
memory usage: 75.6+ KB


In [134]:
import keepdb

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   node_id       1000 non-null   object 
 1   the_int       916 non-null    float64
 2   the_float     903 non-null    float64
 3   the_list      906 non-null    object 
 4   the_dict      908 non-null    object 
 5   the_category  910 non-null    object 
dtypes: float64(2), object(4)
memory usage: 47.0+ KB


In [136]:
keepdb.to_zip('bah.zip', {'df': df})

In [137]:
table = keepdb.pa.from_zip('bah.zip')['df']

In [140]:
df2 = table.to_pandas(types_mapper=pd.ArrowDtype)

In [141]:
df2

,node_id,the_int,the_float,the_list,the_dict,the_category
0,50ae7a81-e1ba-4643-ae25-31abadc56447,<NA>,<NA>,<NA>,"{'A': None, 'B': None, 'C': None, 'D': 0.14768...",E
1,b0d8135d-c0ee-4641-b60a-df4dfc9104a1,<NA>,89.105061,[],"{'A': None, 'B': None, 'C': None, 'D': None}",E
2,c5538f87-d314-4621-9f0a-c646cd3ff60f,826.0,66.310249,['f45914e7-21dd-4e2c-97b5-b989ff674dfc' '2076...,"{'A': None, 'B': 0.9224137956379694, 'C': 0.75...",B
3,c08b2edd-a2f0-41ed-8549-a097f9d33540,434.0,96.997069,['9f1ed1de-af0f-419b-ae17-aeececd64c36' '61d4...,"{'A': None, 'B': None, 'C': 0.8115470088543084...",B
4,a484877e-5664-464d-990f-39427e2588c1,469.0,65.824791,[],"{'A': None, 'B': 0.7253693861027685, 'C': None...",B
...,...,...,...,...,...,...
995,99fbe42f-8a31-483d-9b3c-b30d4e74c3f2,559.0,37.37472,['3b24265c-f133-4053-b5ba-cb6e199b9799'],"{'A': None, 'B': None, 'C': None, 'D': 0.49455...",E
996,7e641aac-7746-43e2-bc34-6e3fb59d4d6e,40.0,62.048404,['6323c800-9338-423c-a802-63d33d885419' 'ddb2...,"{'A': 0.07457206110325909, 'B': None, 'C': 0.8...",C
997,368dd5fd-2f1e-4395-a472-3dcbc064676e,55.0,84.911726,['541df8aa-1d54-4842-8cdf-2c42b13aacb7' 'a2a4...,"{'A': 0.40003854183876986, 'B': None, 'C': Non...",B
998,2c6ea000-5975-4602-bfae-87c59a5b69c1,469.0,24.835409,['0ee90dc8-1c88-4cf8-9268-bcdcd6a7e5e7' '69ec...,"{'A': 0.33276347684993224, 'B': None, 'C': 0.9...",E
